In [6]:
import os
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt



In [7]:
def transform_ctdata(image,windowWidth, windowCenter, normal=False):
    minWindow = float(windowCenter) - 0.5*float(windowWidth)
    newImg = (image - minWindow) / float(windowWidth)
    newImg[newImg < 0] = 0
    newImg[newImg > 1] = 1
    if not normal:
        newImg = (newImg * 255).astype('uint8')
    return newImg

def clahe_equalized(imgs):
    assert(len(imgs.shape)==3)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    imgs_equalized = np.empty(imgs.shape)
    for i in range(len(imgs)):
        imgs_equalized[i,:,:] = clahe.apply(np.array(imgs[i,:,:], dtype=np.uint8))
    return imgs_equalized
    
def plot(orginImage, transimage,originMask,demo, n):
    figure, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 20))
    ax[0].imshow(orginImage[n,:,:],cmap='gray')
    ax[1].imshow(transimage[n,:,:],cmap='gray')
    ax[2].imshow(originMask[n,:,:],cmap='gray')
    ax[3].imshow(demo[n,:,:],cmap='gray')
    figure.tight_layout()
    figure.show()
    
def plot_2d(orginImage,originMask):
    figure, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    ax[0].imshow(orginImage,cmap='gray',vmin=0,vmax=255)
    ax[1].imshow(originMask,cmap='gray',vmin=0,vmax=2)
    figure.tight_layout()
    figure.show()

In [8]:
# 训练集
ct_path = '/home/ps/desktop/hjn/image segmentation/data/tumor/trainImage'
seg_path = '/home/ps/desktop/hjn/image segmentation/data/tumor/trainMask'



outputImg_path = "/home/ps/desktop/hjn/image segmentation/data/LITs_tumor/train_image"
outputMask_path = "/home/ps/desktop/hjn/image segmentation/data/LITs_tumor/train_mask"

# 验证集
# ct_path = '../LITS2017/valid/CT'
# seg_path = '../LITS2017/valid/seg'


# outputImg_path = "data/liver/validImage"
# outputMask_path = "data/liver/validMask"

# 测试集
# ct_path = '../LITS2017/test/CT'
# seg_path = '../LITS2017/test/seg'


# outputImg_path = "data/liver/testImage"
# outputMask_path = "data/liver/testMask"

In [9]:
import PIL.Image as Image
for index, file in enumerate(tqdm(os.listdir(ct_path))):

    # 获取CT图像及Mask数据
    ct_src = sitk.ReadImage(os.path.join(ct_path, file), sitk.sitkInt16)
    mask = sitk.ReadImage(os.path.join(seg_path, file.replace('volume', 'segmentation')), sitk.sitkUInt8)
    # GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    ct_array = sitk.GetArrayFromImage(ct_src)
    mask_array = sitk.GetArrayFromImage(mask)
    
    print(ct_array.shape)
    print(mask_array.shape)

    # mask_array[mask_array == 1] = 0  # 肿瘤
    # mask_array[mask_array == 2] = 1
    
    
    # 阈值截取
#     ct_array[ct_array > 200] = 200
#     ct_array[ct_array < -200] = -200

#     ct_array = ct_array.astype(np.float32)
#     ct_array = ct_array / 200

    # 找到肝脏区域开始和结束的slice，并各向外扩张slice
    z = np.any(mask_array, axis=(1, 2))
    start_slice, end_slice = np.where(z)[0][[0, -1]]

    print('cs',start_slice,end_slice,file)
    start_slice = max(0, start_slice)
    end_slice = min(mask_array.shape[0] - 1, end_slice)

    ct_crop = ct_array[start_slice:end_slice, :, :]
    mask_crop = mask_array[start_slice:end_slice, :, :]

    # #裁剪(偶数才行) 448*448
    # ct_crop = ct_crop[:,32:480,32:480]
    # mask_crop = mask_crop[:,32:480,32:480]

    print('ct_crop.shape',ct_crop.shape)
    
    seg_bg = mask_crop == 0
    seg_liver = mask_crop >= 1
    seg_tumor = mask_crop ==2

    ct_bg = ct_crop * seg_bg
    ct_liver = ct_crop * seg_liver
    ct_tumor = ct_crop * seg_tumor

    liver_min = ct_liver.min()
    liver_max = ct_liver.max()
    tumor_min = ct_tumor.min()
    tumor_max = ct_tumor.max()

    liver_wide = liver_max - liver_min
    liver_center = (liver_max + liver_min)/2
    
    tumor_wide = tumor_max - tumor_min
    tumor_center = (tumor_max + tumor_min)/2
    
    print(tumor_wide,tumor_center)


    ct = transform_ctdata(ct_crop,250,75)
    
    ct_cle = clahe_equalized(ct)

    ct1 = transform_ctdata(ct_crop, liver_wide, liver_center)
    
    ct2 = transform_ctdata(ct_crop, tumor_wide, tumor_center)
    
    ct3 = clahe_equalized(ct2)
    
    ct4 = ct3 / 255
    
    ct5 = 1 - ct4
    
    ct0 = ct_crop.copy()
    
    
    ct_crop[ct_crop > 200] = 200
    ct_crop[ct_crop < -200] = -200

    ct_crop = ct_crop.astype(np.float32)
    ct_crop = ct_crop / 200
    
    frame = 200
    np.save('before.npy', ct0[frame,:,:])
    np.save('after.npy',ct_crop[frame,:,:])
    
    plt.imshow(ct0[frame,:,:],cmap='gray')
    plt.axis('off')
    plt.savefig('before.jpg')
    
    plt.imshow(ct[frame,:,:],cmap='gray')
    plt.axis('off')
    plt.savefig('after.jpg')
    
    
    plot(ct0,ct_crop,ct,ct_cle,frame)

    
    # plot_2d(ct_liver[200,:,:],ct_tumor[200,:,:])
    break

    # 切片处理,并去掉没有病灶的切片
    if int(np.sum(mask_crop))!=0:
        for n_slice in range(mask_crop.shape[0]):
            maskImg = mask_crop[n_slice, :, :]
            if np.max(maskImg) != 2:
                continue
            ctImageArray = np.zeros((ct_crop.shape[1], ct_crop.shape[2], 3), np.float)
            ctImageArray[:, :, 0] = ct_crop[n_slice , :, :]
            ctImageArray[:, :, 1] = ct_crop[n_slice + 1, :, :]
            ctImageArray[:, :, 2] = ct_crop[n_slice + 2, :, :]

            imagepath = outputImg_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"
            maskpath = outputMask_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"

            np.save(imagepath, ctImageArray)  # (448，448,3) np.float dtype('float64')
            np.save(maskpath, maskImg)  # (448，448) dtype('uint8') 值为0 1 2
    else:
        continue
print("Done！")



# ct_array[ct_array > 200] = 200
# ct_array[ct_array < -200] = -200

# ct_array = ct_array.astype(np.float32)
# ct_array = ct_array / 200

# z = np.any(mask_array, axis=(1, 2))
# start_slice, end_slice = np.where(z)[0][[0, -1]]

# print('cs',start_slice,end_slice)
# start_slice = max(0, start_slice)
# end_slice = min(mask_array.shape[0] - 1, end_slice)

# ct_crop = ct_array[start_slice:end_slice, :, :]
# mask_crop = mask_array[start_slice:end_slice, :, :]

# print(ct_crop.shape)
# print(mask_crop.shape)

# maskImg = mask_crop[i, :, :]
            
# ctImageArray = np.zeros((ct_crop.shape[1], ct_crop.shape[2]), np.float)
# ctImageArray[:, :] = ct_crop[i , :, :]
# plot_2d(ctImageArray, maskImg)


# seg_bg = mask_array == 0
# seg_liver = mask_array >= 1
# seg_tumor = mask_array ==2

# ct_bg = ct_array * seg_bg
# ct_liver = ct_array * seg_liver
# ct_tumor = ct_array * seg_tumor

# liver_min = ct_liver.min()
# liver_max = ct_liver.max()
# tumor_min = ct_tumor.min()
# tumor_max = ct_tumor.max()

# liver_wide = liver_max - liver_min
# liver_center = (liver_max + liver_min)/2


# ct = transform_ctdata(ct_array,250,75)

# ct1 = transform_ctdata(ct_array, liver_wide, liver_center)
# print(ct[i,:,:].max())
# print(mask_array[i,:,:].max())
# plot(ct_array,ct_crop,mask_crop,i)










  0%|          | 0/5783 [00:00<?, ?it/s]


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:105:
sitk::ERROR: Unable to determine ImageIO reader for "/home/ps/desktop/hjn/image segmentation/data/tumor/trainImage/59_158.npy"

In [17]:
# 训练集
# ct_path = '../LITS2017/train/CT'
# seg_path = '../LITS2017/train/seg'
# png_path = './png/'

# outputImg_path = "data/tumor/trainImage"
# outputMask_path = "data/tumor/trainMask"

# 验证集
# ct_path = '../LITS2017/valid/CT'
# seg_path = '../LITS2017/valid/seg'
# png_path = './png/'

# outputImg_path = "data/tumor/validImage"
# outputMask_path = "data/tumor/validMask"

# 测试集
# ct_path = '../LITS2017/test/CT'
# seg_path = '../LITS2017/test/seg'
# png_path = './png/'

# outputImg_path = "data/tumor/testImage"
# outputMask_path = "data/tumor/testMask"

In [9]:
# 训练集
# ct_path = '../LITS2017/train/CT'
# seg_path = '../LITS2017/train/seg'
# png_path = './png/'

# outputImg_path = "data/liver/trainImage"
# outputMask_path = "data/liver//trainMask"

# 验证集
ct_path = '../LITS2017/valid/CT'
seg_path = '../LITS2017/valid/seg'
png_path = './png/'

outputImg_path = "data/liver/validImage"
outputMask_path = "data/liver/validMask"

# 测试集
# ct_path = '../LITS2017/test/CT'
# seg_path = '../LITS2017/test/seg'
# png_path = './png/'

# outputImg_path = "data/2d/liver/testImage"
# outputMask_path = "data/2d/liver/testMask"

In [5]:
if not os.path.exists(outputImg_path):
    os.makedirs(outputImg_path)
if not os.path.exists(outputMask_path):
    os.makedirs(outputMask_path)

def file_name_path(file_dir, dir=True, file=False):
    """
    get root path,sub_dirs,all_sub_files
    :param file_dir:
    :return: dir or file
    """
    for root, dirs, files in os.walk(file_dir):
        if len(dirs) and dir:
            print("sub_dirs:", dirs)
            return dirs
        if len(files) and file:
            print("files:", files)
            return files

def crop_ceter(img, croph, cropw):
    #for n_slice in range(img.shape[0]):
    height, width = img[0].shape
    starth = height//2 - (croph//2)
    startw = width//2 - (cropw//2)
    return img[:, starth:starth+croph, startw:startw+cropw]


for index, file in enumerate(tqdm(os.listdir(ct_path))):

    # 获取CT图像及Mask数据
    ct_src = sitk.ReadImage(os.path.join(ct_path, file), sitk.sitkInt16)
    mask = sitk.ReadImage(os.path.join(seg_path, file.replace('volume', 'segmentation')), sitk.sitkUInt8)
    # GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    ct_array = sitk.GetArrayFromImage(ct_src)
    mask_array = sitk.GetArrayFromImage(mask)
    
    
#     seg_bg = mask_array == 0
#     seg_liver = mask_array >= 1
#     seg_tumor = mask_array ==2

#     ct_bg = ct_array * seg_bg
#     ct_liver = ct_array * seg_liver
#     ct_tumor = ct_array * seg_tumor

    # liver_min = ct_liver.min()
    # liver_max = ct_liver.max()
    # tumor_min = ct_tumor.min()
    # tumor_max = ct_tumor.max()

    # liver_wide = liver_max - liver_min
    # liver_center = (liver_max + liver_min)/2
    
#     tumor_wide = tumor_max - tumor_min
#     tumor_center = (tumor_max + tumor_min)/2
    
#     ct = transform_ctdata(ct_array, tumor_wide, tumor_center,True)
    
#     ct = clahe_equalized(ct)

#     mask_array[mask_array == 1] = 0  # 肿瘤

#     mask_array[mask_array == 2] = 1

    # 阈值截取
    ct_array[ct_array > 200] = 200
    ct_array[ct_array < -200] = -200

    ct_array = ct_array.astype(np.float32)
    ct_array = ct_array / 200

    # 找到肝脏区域开始和结束的slice，并各向外扩张slice
#     z = np.any(mask_array, axis=(1, 2))
#     start_slice, end_slice = np.where(z)[0][[0, -1]]

#     print('cs',start_slice,end_slice,file)
#     start_slice = max(0, start_slice - 1)
#     end_slice = min(mask_array.shape[0] - 1, end_slice + 2)

#     ct_crop = ct_array[start_slice:end_slice, :, :]
#     mask_crop = mask_array[start_slice+1:end_slice-1, :, :]
    z = np.any(mask_array, axis=(1, 2))
    start_slice, end_slice = np.where(z)[0][[0, -1]]

    print('cs',start_slice,end_slice,file)
    start_slice = max(0, start_slice)
    end_slice = min(mask_array.shape[0] - 1, end_slice)

    ct_crop = ct_array[start_slice:end_slice, :, :]
    mask_crop = mask_array[start_slice:end_slice, :, :]

    print('ct_crop.shape',ct_crop.shape)
    
    
    #裁剪(偶数才行) 448*448
    ct_crop = ct_crop[:,32:480,32:480]
    mask_crop = mask_crop[:,32:480,32:480]

    print('ct_crop.shape',ct_crop.shape)
    # 切片处理,并去掉没有病灶的切片
    if int(np.sum(mask_crop))!=0:
        for n_slice in range(mask_crop.shape[0]):
            maskImg = mask_crop[n_slice, :, :]
            # if np.max(maskImg) != 2:
            #     continue
            # ctImageArray = np.zeros((ct_crop.shape[1], ct_crop.shape[2], 3), np.float)
            # ctImageArray[:, :, 0] = ct_crop[n_slice , :, :]
            # ctImageArray[:, :, 1] = ct_crop[n_slice + 1, :, :]
            # ctImageArray[:, :, 2] = ct_crop[n_slice + 2, :, :]
            
            ctImageArray = np.zeros((ct_crop.shape[1], ct_crop.shape[2], 1), np.float)
            ctImageArray[:, :,0] = ct_crop[n_slice , :, :]

            imagepath = outputImg_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"
            maskpath = outputMask_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"

            np.save(imagepath, ctImageArray)  # (448，448,3) np.float dtype('float64')
            np.save(maskpath, maskImg)  # (448，448) dtype('uint8') 值为0 1 2
    else:
        continue
print("Done！")

  0%|          | 0/5783 [00:00<?, ?it/s]


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:105:
sitk::ERROR: Unable to determine ImageIO reader for "/home/ps/desktop/hjn/image segmentation/data/tumor/trainImage/59_158.npy"

In [ ]:
# 切片为3

if not os.path.exists(outputImg_path):
    os.makedirs(outputImg_path)
if not os.path.exists(outputMask_path):
    os.makedirs(outputMask_path)

def file_name_path(file_dir, dir=True, file=False):
    """
    get root path,sub_dirs,all_sub_files
    :param file_dir:
    :return: dir or file
    """
    for root, dirs, files in os.walk(file_dir):
        if len(dirs) and dir:
            print("sub_dirs:", dirs)
            return dirs
        if len(files) and file:
            print("files:", files)
            return files

def crop_ceter(img, croph, cropw):
    #for n_slice in range(img.shape[0]):
    height, width = img[0].shape
    starth = height//2 - (croph//2)
    startw = width//2 - (cropw//2)
    return img[:, starth:starth+croph, startw:startw+cropw]


for index, file in enumerate(tqdm(os.listdir(ct_path))):

    # 获取CT图像及Mask数据
    ct_src = sitk.ReadImage(os.path.join(ct_path, file), sitk.sitkInt16)
    mask = sitk.ReadImage(os.path.join(seg_path, file.replace('volume', 'segmentation')), sitk.sitkUInt8)
    # GetArrayFromImage()可用于将SimpleITK对象转换为ndarray
    ct_array = sitk.GetArrayFromImage(ct_src)
    mask_array = sitk.GetArrayFromImage(mask)

    # mask_array[mask_array == 1] = 0  # 肿瘤
    mask_array[mask_array == 2] = 1

    # 阈值截取
    ct_array[ct_array > 200] = 200
    ct_array[ct_array < -200] = -200

    ct_array = ct_array.astype(np.float32)
    ct_array = ct_array / 200

    # 找到肝脏区域开始和结束的slice，并各向外扩张slice
#     z = np.any(mask_array, axis=(1, 2))
#     start_slice, end_slice = np.where(z)[0][[0, -1]]

#     print('cs',start_slice,end_slice,file)
#     start_slice = max(0, start_slice - 1)
#     end_slice = min(mask_array.shape[0] - 1, end_slice + 2)

#     ct_crop = ct_array[start_slice:end_slice, :, :]
#     mask_crop = mask_array[start_slice+1:end_slice-1, :, :]

    #裁剪(偶数才行) 448*448
    ct_crop = ct_crop[:,32:480,32:480]
    mask_crop = mask_crop[:,32:480,32:480]

    print('ct_crop.shape',ct_crop.shape)

    # 切片处理,并去掉没有病灶的切片
    if int(np.sum(mask_crop))!=0:
        for n_slice in range(mask_crop.shape[0]):
            maskImg = mask_crop[n_slice, :, :]
            if np.max(maskImg) != 2:
                continue
            ctImageArray = np.zeros((ct_crop.shape[1], ct_crop.shape[2], 3), np.float)
            ctImageArray[:, :, 0] = ct_crop[n_slice , :, :]
            ctImageArray[:, :, 1] = ct_crop[n_slice + 1, :, :]
            ctImageArray[:, :, 2] = ct_crop[n_slice + 2, :, :]

            imagepath = outputImg_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"
            maskpath = outputMask_path + "/" + str(index+1) + "_" + str(n_slice) + ".npy"

            np.save(imagepath, ctImageArray)  # (448，448,3) np.float dtype('float64')
            np.save(maskpath, maskImg)  # (448，448) dtype('uint8') 值为0 1 2
    else:
        continue
print("Done！")

In [ ]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

onServer = False
if onServer:
    niiSegPath = './LITS17/seg/'
    niiImagePath = './LITS17/ct/'
else:
    niiSegPath = '~/LITS17/seg/'
    niiImagePath = '~/LITS17/ct/'

def getRangeImageDepth(image):
    z = np.any(image, axis=(1,2)) # z.shape:(depth,)
    #print("all index:",np.where(z)[0])
    if len(np.where(z)[0]) >0:
        startposition,endposition = np.where(z)[0][[0,-1]]
    else:
        startposition = endposition = 0

    return startposition, endposition
"""
会画出每个病人肿瘤区域最大切片的直方图
与汇总的直方图
"""
total_liver = []
total_tumor = []
colors = ['b','g']
for i in range(0, 131, 1):
    seg = sitk.ReadImage(niiSegPath+ "segmentation-" + str(i) + ".nii", sitk.sitkUInt8)
    segimg = sitk.GetArrayFromImage(seg)
    src = sitk.ReadImage(niiImagePath+"volume-" + str(i) + ".nii")
    srcimg = sitk.GetArrayFromImage(src)

    seg_liver = segimg.copy()
    seg_liver[seg_liver>0] = 1

    seg_tumorimage = segimg.copy()
    seg_tumorimage[segimg == 1] = 0
    seg_tumorimage[segimg == 2] = 1

    # 获取含有肿瘤切片的起、止位置
    start,end = getRangeImageDepth(seg_tumorimage)
    if start==0 and end == 0:
        print("continue")
        continue
    print("start:",start," end:",end)

    max_tumor=0 # 记录肿瘤的最大占比
    max_tumor_index = -1 # 记录最大肿瘤所在切片


    for j in range(start, end+1):
        if np.mean(seg_tumorimage[j]) > max_tumor:
            max_tumor = np.mean(seg_tumorimage[j])
            max_tumor_index = j

#        batch_image.append(srcimg[j,:,:])
#        batch_mask.append(seg_tumorimage[j,:,:])

    src_flatten = srcimg[max_tumor_index].flatten()
    liver_flatten = seg_liver[max_tumor_index].flatten()
    tumor_flatten = seg_tumorimage[max_tumor_index].flatten()

    liver = [] # liver hu value
    tumor = []
    for j in range(src_flatten.shape[0]):
        if liver_flatten[j]>0:
            liver.append(src_flatten[j])
        if tumor_flatten[j]>0:
            tumor.append(src_flatten[j])

    total_liver.append(liver)
    total_tumor.append(tumor)
    """
    # 因为肝脏区域很多而肿瘤很少，所以画直方图使用相同的y-axis就会导致
    # 几乎看不到肿瘤的直方图
    plt.hist(flat_total_liver, color = "skyblue", bins=100, alpha=0.5, 	    label='liver hu')
    plt.hist(flat_total_tumor, color = "red", bins=100, alpha=0.5, label='tumor hu')
    plt.legend(loc='upper right')
    """
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.hist([liver, tumor], bins=100,color=colors)
    n, bins, patches = ax1.hist([liver,tumor], bins=100)
    ax1.cla() #clear the axis

    #plots the histogram data
    width = (bins[1] - bins[0]) * 0.4
    bins_shifted = bins + width
    ax1.bar(bins[:-1], n[0], width, align='edge', color=colors[0])
    ax2.bar(bins_shifted[:-1], n[1], width, align='edge', color=colors[1])

    #finishes the plot
    ax1.set_ylabel("liver hu Count", color=colors[0])
    ax2.set_ylabel("tumor hu Count", color=colors[1])
    ax1.tick_params('y', colors=colors[0])
    ax2.tick_params('y', colors=colors[1])
    plt.tight_layout()
    plt.title("person[%d],slice[%d]"%(i,max_tumor_index))
    plt.savefig("LITS/person%d_slice%d.png"%(i,max_tumor_index))
    plt.clf()

# 用来展平total_liver和total_tumor里面的值
flat_total_liver = [item for sublist in total_liver for item in sublist]
flat_total_tumor = [item for sublist in total_tumor for item in sublist]
#plt.hist(flat_total_liver, color = "skyblue", bins=100, alpha=0.5, label='liver hu')
#plt.hist(flat_total_tumor, color = "red", bins=100, alpha=0.5, label='tumor hu')
#plt.legend(loc='upper right')
#plt.title("total hu")
#plt.savefig("LITS/total.png")
#plt.clf()


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.hist([flat_total_liver, flat_total_tumor], bins=100,color=colors)
n, bins, patches = ax1.hist([flat_total_liver,flat_total_tumor], bins=100)
ax1.cla() #clear the axis

#plots the histogram data
width = (bins[1] - bins[0]) * 0.4
bins_shifted = bins + width
ax1.bar(bins[:-1], n[0], width, align='edge', color=colors[0])
ax2.bar(bins_shifted[:-1], n[1], width, align='edge', color=colors[1])

#finishes the plot
ax1.set_ylabel("liver hu Count", color=colors[0])
ax2.set_ylabel("tumor hu Count", color=colors[1])
ax1.tick_params('y', colors=colors[0])
ax2.tick_params('y', colors=colors[1])
plt.tight_layout()
plt.title("total")
plt.savefig("LITS/total.png")
plt.clf()